In [ ]:
import util, scrape

import warnings
warnings.filterwarnings("ignore")

from tqdm.auto import tqdm
tqdm.pandas()

import collections
import itertools
import re
import pickle
import csv
import multiprocessing
import operator
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import seaborn as sns

import gensim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_fscore_support

from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

In [ ]:
# load trip reports dataframe
with open(util.TRIP_REPORTS_DATAFRAME_FILE, "rb") as f:
    df = pickle.load(f)
    
df

In [ ]:
# label docs individually by enumeration
def label_docs(X):
    labeled = []
    for i, doc in enumerate(X):
        labeled.append(gensim.models.doc2vec.TaggedDocument(words=doc, tags=[f"DOC_{i}"]))
    return labeled


In [ ]:
# create mask for dataframe columns to only include:
# (1) top N drugs with the most trip reports 
# (2) trip reports with at least this many tokens to ensure informative content

N = 10 
MIN_TOKENS = 30 

counter = collections.Counter(df["drug"])
top_drugs = sorted(counter, key=lambda x: counter[x], reverse=True)[:N]

mask = [True if (len(tokens) > MIN_TOKENS) and (df["drug"][i] in top_drugs) else False for i, tokens in enumerate(df["trip_report_tokenized"])]


In [ ]:
def get_train_test_split_indices(X, y, train_size=0.8):
    indices = [i for i in range(len(X))]
    train_indices, test_indices, y_train, y_test = train_test_split(indices, y, train_size=train_size, stratify=y)
    
    return train_indices, test_indices
    
def split_X_into_train_test_sets(X, train_indices, test_indices):
    X_train = np.zeros((len(train_indices), model.vector_size))
    X_test = np.zeros((len(test_indices), model.vector_size))
    for i, train_index in enumerate(train_indices):
        X_train[i] = X[train_index]
    for i, test_index in enumerate(test_indices):
        X_test[i] = X[test_index]
        
    return X_train, X_test

def split_y_into_train_test_sets(y, train_indices, test_indices):
    y_train = [y[train_index] for train_index in train_indices]
    y_test = [y[test_index] for test_index in test_indices]
    
    return y_train, y_test
    

In [ ]:
train_indices, test_indices = get_train_test_split_indices(X, y, train_size=0.8)
y = df["drug"][mask]
y_train, y_test = split_y_into_train_test_sets(y, train_indices, test_indices)

In [ ]:
# train doc2vec model by num_epochs
def train_doc2vec_model(model, docs, num_epochs, lr_step, lr_min=0.0, verbose=True):
    for epoch in range(1, num_epochs+1):
        if verbose: print(f"Epoch: {epoch}")
        model.train(docs, total_examples=model.corpus_count, epochs=1)
        model.alpha -= lr_step  # decrease the learning rate
        model.min_alpha = lr_min
        

In [ ]:
# test given classifier on test set
def test_classifier(clf, X_test, y_test, labels, top_n=3, show_top_n_accuracy=False, show_confusion_matrix=False):
    y_pred = clf.predict(X_test)
    precision, recall, f_score, support = precision_recall_fscore_support(y_test, y_pred, labels=labels, average=None)
    report = pd.DataFrame({
        "class": labels,
        "precision": precision,
        "recall": recall,
        "f_score": f_score,
        "support": support
    })
    
    if show_top_n_accuracy:
        y_hat = clf.predict_proba(X_test)
        classes = clf.classes_
        y_pred_sorted = []
        y_hat_sorted = []
        num_correct = 0
        for i, probs in enumerate(y_hat):
            probs_sorted, classes_sorted = (list(l) for l in zip(*sorted(zip(probs, classes), key=operator.itemgetter(0), reverse=True)))
            true_class = y_test.iloc[i]
            top_classes = classes_sorted[:top_n]
            top_probs = probs_sorted[:top_n]
            y_pred_sorted.append(classes_sorted)
            y_hat_sorted.append(probs_sorted)
            if true_class in top_classes:
                num_correct += 1
        top_n_acc = num_correct / len(y_pred)
        print(f"Top-{top_n} accuracy: {top_n_acc}")
    
    if show_confusion_matrix:
        cm = confusion_matrix(y_test, y_pred)
        new_cm = []
        for row in cm:
            support = sum(row)
            new_row = row / support
            new_cm.append(new_row)
        cm = new_cm

        df_cm = pd.DataFrame(cm, index=labels, columns=labels)
        plt.figure(figsize=(20,20))
        ax = sns.heatmap(df_cm, annot=False, linewidth=0.05)
        bottom, top = ax.get_ylim()
        ax.set_ylim(bottom + 0.5, top - 0.5)
        
    return report


In [ ]:
vector_sizes = [64, 128] 
windows = [3, 5, 7, 9] 
lrs = [7.5e-2, 2.5e-2, 7.5e-3]
lr_steps = [1.0e-3, 3.0e-3, 5.0e-5, 0.0]

docs = label_docs(df["trip_report_tokenized"][mask])

best_f_score_avg = 0.0
best_model = None
best_hyperparams = None
best_report = None
for vector_size, window, lr, lr_step in itertools.product(vector_sizes, windows, lrs, lr_steps):
    # If dm=1, ‘distributed memory’ (PV-DM) is used. Otherwise, distributed bag of words (PV-DBOW) is employed.
    model = gensim.models.doc2vec.Doc2Vec(docs, workers=multiprocessing.cpu_count(), dm=0, vector_size=vector_size, window=window, alpha=lr, min_alpha=lr_min)
    epochs_step = 5
    epochs_trained = 0
    for i in range(10):
        train_doc2vec_model(model, docs, epochs_step, lr_step)
        epochs_trained += epochs_step
    
        # split X into train and test sets using established indices
        X = model.docvecs
        X_train, X_test = split_X_into_train_test_sets(X, train_indices, test_indices)
        
        # train classifier
        clf = GaussianNB()
        clf.fit(X_train, y_train)
        
        # test classifier and save best model
        labels = np.unique(y_train)
        report = test_classifier(clf, X_test, y_test, labels)
        f_score_avg = np.mean(report["f_score"])
        if f_score_avg >= best_f_score_avg:
            best_f_score_avg = f_score_avg
            best_model = copy.deepcopy(model)
            best_hyperparams = (epochs_trained, vector_size, window, lr, lr_step)
            best_report = report

print(f"Best model: ")
print(f"\tEpochs trained: {epochs_trained}")
print(f"\tVector size: {vector_size}")
print(f"\tWindow size: {window}")
print(f"\tLearning rate: {lr}")
print(f"\tLearning rate step decrease: {lr_step}")
print(f"\tAverage F-score: {best_f_score_avg}")
print("\tClassification report:\n")
print(best_report)
print("")

model = best_model
epochs_trained, vector_size, window, lr, lr_step = best_hyperparams

In [ ]:
# save best model
with open(util.DOC2VEC_MODEL_FILE, "wb") as f:
    pickle.dump(best_model, f)
    
# save best hyperparameters
with open(util.DOC2VEC_HYPERPARAMETERS_FILE, "w") as f:
    
    f.write(f"Epochs trained: {epochs_trained}\n")
    f.write(f"Vector size: {vector_size}\n")
    f.write(f"Window size: {window}\n")
    f.write(f"Learning rate: {lr}\n")
    f.write(f"Learning rate step decrease: {lr_step}\n")


In [ ]:
# run grid search on classifiers and provided parameters and return the best result
def grid_search_classifiers(X_train, X_test, y_train, y_test, clfs, clf_to_gs_params_dict, cv=5):
    reports = []
    f_score_avgs = []
    for clf in clfs:
        print(f"\tTraining classifier: {clf.__class__}...")
        gs = GridSearchCV(clf, clf_to_gs_params_dict[clf.__class__], cv=cv)
        gs.fit(X_train, y_train)
        labels = np.unique(y_train)
        report = test_classifier(gs, X_test, y_test, labels)
        reports.append(report)
        f_score_avg = np.mean(report["f_score"])
        f_score_avgs.append(f_score_avg)
        
    results_sorted = sorted(zip(clfs, f_score_avgs, reports), key=lambda x: x[1], reverse=True)
    best_result = results_sorted[0]
            
    return best_result


In [ ]:
# run grid search wrapped in a k-fold
def grid_search_classifiers_k_fold(X, y, clfs, clf_to_gs_params_dict, k_fold=10):

    results = []
    skf = StratifiedKFold(n_splits=k_fold)
    train_test_split_indices = skf.split(X, y)
    
    for i, (train_indices, test_indices) in enumerate(train_test_split_indices):

        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]
        
        result = grid_search_classifiers(X_train, X_test, y_train, y_test, clfs, clf_to_gs_params_dict)
        results.append(result)
        
    return all_results


In [ ]:
clfs = [
    GaussianNB(),
    LogisticRegression(),
    KNeighborsClassifier(),
    SVC(probability=True),
    MLPClassifier()
]

clf_to_gs_params_dict = {
    GaussianNB().__class__: {
    },
    LogisticRegression().__class__: {
        "penalty": ("l1", "l2"),
        "C": (1.0e0, 1.0e-1, 1.0e-2, 1.0e-3, 1.0e-4, 1.0e-5)
    },
    KNeighborsClassifier().__class__: {
        "n_neighbors": (4, 8, 12)
    },
    SVC().__class__: {
        "kernel": ("rbf", "poly"),
        "gamma": (1.0e0, 1.0e1, 1.0e2, 1.0e3),
        "degree": (2, 4, 6)
        
    },
    MLPClassifier().__class__: {
        "alpha": (1.0e-3, 1.0e-4, 1.0e-5),
        "batch_size": (8, 16, 64),
        "learning_rate": ("constant", "adaptive"),
        "learning_rate_init": (1.0e-2, 1.0e-3, 1.0e-4),
        "tol": (1e-3, 1e-4, 1e-4),
        "early_stopping": (True),
        "momentum": (0.5, 0.9),
    }
}

# split X into train and test sets using established indices
X = model.docvecs
X_train, X_test = split_X_into_train_test_sets(X, train_indices, test_indices)

best_clf, best_f_score_avg, best_report = grid_search_classifiers(X_train, X_test, y_train, y_test, clfs, clf_to_gs_params_dict)
print(f"\tBest classifier: {best_clf}")
print(f"\tAverage F-score: {best_f_score_avg}")
print("\tClassification report:\n")
print(best_report)
print("")


In [ ]:
drugs = sorted(np.unique(y))


In [ ]:
# Fit the model using t-SNE randomized algorithm
X_projected = TSNE(random_state=0).fit_transform(model.docvecs.vectors_docs)


In [ ]:
def scatter(X, drugs_to_plot=drugs):
    # make a custom color palette with seaborn
    gray_palette = [[0.8, 0.8, 0.8]]
    color_palette = sns.color_palette("hls", len(drugs_to_plot))
    palette = np.array(color_palette + gray_palette)
    sns.palplot(palette)
    
    # get colors for selected classes
    drug_to_rank_dict = {drug: i for i, drug in enumerate(drugs_to_plot)}
    color_indices = np.array([np.int(drug_to_rank_dict[drug]) if drug in drugs_to_plot else -1 for drug in y])
    color_index_to_drug_dict = {i: drug for i, drug in enumerate(drugs_to_plot)}

    # We create a scatter plot.
    f = plt.figure(figsize=(32, 32))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(X[:,0], X[:,1], lw=0, s=120,
                    c=palette[color_indices])
    #plt.xlim(-25, 25)
    #plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # We add the labels for each cluster.
    
    txts = []
    for i, drug in enumerate(drugs_to_plot):
        # Position of each label.
        xtext, ytext = np.median(X[color_indices == i, :], axis=0)
        txt = ax.text(xtext, ytext, color_index_to_drug_dict[i], fontsize=20)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)
    
    return f, ax, sc, txts


In [ ]:
scatter(X_projected)


In [ ]:
for drug in drugs:
    drugs_to_plot = [drug]
    scatter(X_projected, drugs_to_plot)
    

In [ ]:
# get average doc vec
avg_doc_vec = np.mean([model.docvecs[i] for i in range(len(model.docvecs))], axis=0)

# accumulate doc vecs for each drug in a dictionary
drug_to_doc_vecs_dict = collections.defaultdict(list)
for i, drug in enumerate(df["drug"]):
    doc_tag = f"DOC_{i}"
    doc_vec = model.docvecs[doc_tag]
    drug_to_doc_vecs_dict[drug].append(doc_vec)
    
# assign a psych vec for each psychedelic by taking adding the difference between the mean of the drug's doc vecs 
# and the average doc vec to the mean of the drug's doc vecs, in order to produce a vector that is more extreme in
# the ways that the drug's average doc vec is already extreme, so as to highlight what makes it distinct from the 
# other drug's average doc vec
SIGMA = 0
drug_to_psych_vec_dict = {drug : np.mean(doc_vecs, axis=0) + ((np.mean(doc_vecs, axis=0) - avg_doc_vec) * SIGMA) for drug, doc_vecs in drug_to_doc_vecs_dict.items()}

# print the words whose vectors are closest to each psych vec
for drug, psych_vec in drug_to_psych_vec_dict.items():
    print(drug+'\n', model.wv.most_similar([psych_vec]), '\n')
    